In [37]:
from selenium import webdriver
from time import sleep

driver = webdriver.Chrome('ChromeDriver/chromedriver')
driver.set_page_load_timeout(2)

try:
    driver.get("https://jewelry.ha.com/c/search-results.zx?N=776+790+231+2040&Ne=2014&erpp=204")
except:
    print("time out")

time out


In [120]:
Condition = True
# URL_Links = []

while Condition:

    sleep(3)
    search_result = driver.find_elements_by_class_name('item-title')
    search_result = [link.get_attribute('href') for link in search_result]
    URL_Links.append(search_result)
    try:
        driver.find_element_by_xpath('//*[@id="tabs-1"]/div[3]/a[12]').click()
        sleep(8)
    except:
        Condition = False


In [121]:
URL_Link = []
for links in URL_Links:
    for link in search_result:
        URL_Link.append(link)
print(len(URL_Link))

4477


In [123]:
import json
#saving list of links into a json file
url = {'links' : URL_Link}
with open('url.json', 'w') as jsonfile:
    json.dump(url, jsonfile)

In [2]:
import json
with open('url.json') as json_file:
    url = json.load(json_file)

In [3]:
url_link = url['links']

In [12]:
import pandas as pd
diamond_ring_data_2 = pd.DataFrame()
Ring = {}

for link in url_link:
    try:
        driver.get(link)
        sleep(3)
    except:
        print('TIME OUT')

    sleep(5)
    try:
        ring_price = driver.find_element_by_xpath('//*[@id="page-content"]/div/div[5]/div[1]/div[3]/div/div/small[2]').text
    except:
        ring_price = None
    try:
        description = driver.find_element_by_xpath('//*[@id="auction-description"]/span').text
    except:
        description = None
    try:
        condition_report = driver.find_element_by_xpath('//*[@id="condition-report"]').text
    except:
        condition_report = None

    ring = {
        'Ring Price': ring_price,
        'Description': description,
        'Condition Report' : condition_report
    }
    print(ring)
    diamond_ring_data_2 = diamond_ring_data_2.append(ring, ignore_index=True)
    diamond_ring_data_2.to_csv('diamond_ring_data_3.csv')


TIME OUT
{'Ring Price': 'SOLD ON DEC 6, 2011 FOR: $507.88', 'Description': 'Diamond, White Gold Ring\nThe ring features full-cut diamonds weighing a total of approximately 0.15 carat, set in 18k white gold, Italian hallmarks. Gross weight 12.50 grams.\nSize: 6 (not sizeable)', 'Condition Report': 'Condition Report*:\nDiamond Average Color: G-H-I\nDiamond Average Clarity: SI\nDiamond Shapes: Full-cut\nDiamond Count: 1 - 25\nOverall Condition: Excellent'}
TIME OUT
{'Ring Price': 'SOLD ON MAY 3, 2011 FOR: $507.88', 'Description': "Carved Opal, Diamond, Gold Ring\nThe ring features a carved opal depicting a lady's profile measuring 38.00 x 18.00 mm, accented by a full-cut diamond weighing approximately 0.12 carat, set in 14k gold. Gross weight 18.70 grams.\n*Note: Size 7-1/4 (sizeable)\n\nThe Property of the Mary Grace Horlock Estate", 'Condition Report': 'Condition Report*:\nDiamond Average Color: J-K-L\nDiamond Average Clarity: VS\nDiamond Shapes: Full-cut\nDiamond Count: 1 - 25\nGem Typ

In [1]:
import pandas as pd
import re

data_frame = pd.read_csv('..\Diamond_Rings_data\daimond_ring_data.csv')


In [2]:
pf = pd.DataFrame(data_frame)
pf.head()

,Unnamed: 0,Condition Report,Description,Sold Price
0,0,Condition Report*:\nDiamonds:\nAvg Color: G-H-...,"Diamond, Platinum Ring\nThe ring features a ma...","SOLD ON OCT 5, 2020 FOR: $40,000.00"
1,1,Condition Report*:\nDiamonds:\nAvg Color: G-H-...,"Diamond, White Gold Ring\nThe ring features a ...","SOLD ON DEC 8, 2019 FOR: $40,000.00"
2,2,Condition Report*:\nDiamonds:\nAvg Color: M+\n...,"Diamond, Platinum, White Gold Ring, French\nTh...","SOLD ON JUN 26, 2017 FOR: $40,000.00"
3,3,Condition Report*:\nDiamonds:\nAvg Color: M+\n...,"Diamond, Platinum Ring\nThe ring features a Eu...","SOLD ON APR 3, 2017 FOR: $40,000.00"
4,4,Condition Report*:\nDiamond Average Color: G-H...,"Fancy Light Yellow Diamond, Diamond, Gold Ring...","SOLD ON APR 19, 2016 FOR: $40,000.00"


In [14]:
m = pf['Condition Report']
m.head()

0    Condition Report*:\nDiamonds:\nAvg Color: G-H-...
1    Condition Report*:\nDiamonds:\nAvg Color: G-H-...
2    Condition Report*:\nDiamonds:\nAvg Color: M+\n...
3    Condition Report*:\nDiamonds:\nAvg Color: M+\n...
4    Condition Report*:\nDiamond Average Color: G-H...
Name: Condition Report, dtype: object

In [15]:
m = m.str.split('\n')


In [17]:
m[110]

['Condition Report*:',
 'Diamonds:',
 'Avg Color: G-H-I',
 'Avg Clarity: SI',
 'Shape(s): Triangular',
 'Count: 1 - 25',
 '',
 'Certified Diamonds:',
 'Stone: 1',
 'Lab: GIA',
 'Certification #: 2173379507',
 'Carat: 3.09',
 'Color: I',
 'Clarity: VS1',
 'Shape: Emerald-cut',
 '',
 'Metals:',
 'Platinum',
 '',
 'Overall Condition: Very Good',
 'Condition Notes: Stamped for platinum. A gorgeous, classic ring!']

In [315]:

def get_diamond_count(x):
    for item in x:
        if 'Gem' not in item:
            if 'Count' in item and ':' in item and '-'in item:
                mask = item.split('-')[1]
                return int(mask)
            elif 'Count' in item and '-' in item:
                return int(item.split('-')[1])
            elif 'Count' in item and ':' in item and '+' in item:
                mask = item.split(':')[1]
                mask = mask.split('+')[0]
                return int(mask)
            elif 'Count' in item and '+' in item:
                mask = item.split('Count')[1]
                mask = mask.split('+')[0]
                return int(mask)
            else:
                pass
        else:
            pass
        


In [316]:
diamond_count = m.apply(get_diamond_count)

In [7]:
# pd.set_option('display.max_rows', None)

In [317]:

diamond_count.head()


0    25.0
1    50.0
2    25.0
3    25.0
4    25.0
Name: Condition Report, dtype: float64

In [118]:
m1 = pf['Sold Price']
m1.head()

0     SOLD ON OCT 5, 2020 FOR: $40,000.00
1     SOLD ON DEC 8, 2019 FOR: $40,000.00
2    SOLD ON JUN 26, 2017 FOR: $40,000.00
3     SOLD ON APR 3, 2017 FOR: $40,000.00
4    SOLD ON APR 19, 2016 FOR: $40,000.00
Name: Sold Price, dtype: object

In [26]:
def clean_price(x):
    for item in x:
        mask = x.split('$')[1]
        mask = mask.replace(',', '')
        mask = mask.replace('.', '')
        return float(mask)

In [298]:
diamond_ring_price = m1.apply(clean_price)
diamond_ring_price.head()

0    4000000.0
1    4000000.0
2    4000000.0
3    4000000.0
4    4000000.0
Name: Sold Price, dtype: float64

In [281]:
m2= pf['Description']
m2.head()

0    Diamond, Platinum Ring\nThe ring features a ma...
1    Diamond, White Gold Ring\nThe ring features a ...
2    Diamond, Platinum, White Gold Ring, French\nTh...
3    Diamond, Platinum Ring\nThe ring features a Eu...
4    Fancy Light Yellow Diamond, Diamond, Gold Ring...
Name: Description, dtype: object

In [282]:
m2 = m2.str.split('\n')

In [151]:
def get_gross_weight(x):
    for item in x:
        if 'Gross weight' in item:
            mask = item.split('Gross weight')[1]
            mask = mask.split('grams')[0]
            if 'is' in mask:
                return float(mask.split('is')[1])
            else:
                return float(mask)
        else:
            pass

In [286]:
def get_size(x):
    for item in x:
        if 'Size:' in item and '(sizeable)' in item :
            mask = item.split('Size:')[1]
            mask = mask.split('(sizeable)')[0]
            if '-' in mask:
                mask = mask.split('-')[0]
                return mask
            elif '(sizeable)' in mask:
                mask =  mask.split('(sizeable)')[0]
                return mask
            elif 's:' in mask:
                mask = mask.split('s')[1]
                return mask
            else:
                pass
            return mask
        elif 'Size' in item and '(sizeable)' in item:
            mask = item.split('Size')[1]
            mask.split('(sizeable)')[0]
            if '-' in mask:
                mask = mask.split('-')[0]
                return mask
            elif '(sizeable)' in mask:
                mask =  mask.split('(sizeable)')[0]
                return mask
            elif 's:' in mask:
                mask = mask.split('s')[1]
                return mask
            else:
                pass
            return mask
        if 'Size :' in item and '(sizeable)' in item:
            mask = item.split('Size :')[1]
            mask = mask.split('(sizeable)')[0]
            if '-' in mask:
                mask = mask.split('-')[0]
                return mask
            elif '(sizeable)' in mask:
                mask =  mask.split('(sizeable)')[0]
                return mask
            elif 's:' in mask:
                mask = mask.split('s')[1]
                return mask
            else:
                pass
            return mask
        else:
            pass


In [299]:
ring_gross_weight = m2.apply(get_gross_weight)
ring_gross_weight.head()

0    5.10
1    6.90
2    4.47
3    5.51
4    5.90
Name: Description, dtype: float64

In [300]:
ring_size = m2.apply(get_size)


In [301]:
ring_size = pd.to_numeric(size, errors='coerce')
ring_size.head()

0    6.0
1    6.0
2    4.0
3    6.0
4    6.0
Name: Description, dtype: float64

In [65]:
m3 = pf['Description']

In [8]:
m3 = m3.str.split('\n')

In [66]:
m3[1110]

'Ruby, Diamond, Platinum, Gold Ring\nThe ring features a cushion-shaped ruby measuring 10.10 x 9.60 x 4.00 mm and weighing approximately 3.60 carats, enhanced by full-cut diamonds weighing a total of approximately 1.55 carats, accented by tapered baguette-cut diamonds weighing a total of approximately 1.20 carats, set in platinum, completed by an 18k gold shank. Total diamond weight is approximately 2.75 carats. Gross weight 13.75 grams.\n*Note: Size 6 (sizeable)\n\nThe Property of the Estate of John R. Archer'

In [52]:
Diamond_Rings_data = {
    'Ring Metal' : ['Platinum'],
    'Diamonds shape in each Ring' : ['emerald-cut diamond, triangular-shaped diamonds'],
    'Gems shape if any' :[None],
    'Measurments in mm' : ['11.48 , 7.29 , 4.08'],
    'Diamond Weights in Carats' : [' 3.09 , 1.50'],
    'Gem Weight in Carat' : [None],
    'Diamond Avg Color' : ['G-H-I'],
    'Diamond Avg Clarity' : ['SI'],
    'Gem Type if any' : [None],
    'Gem Average Clarity' : [None],
    'Gem Min Count': [None],
    'Gem Max Count': [None],
    'Diamond Min Count' : ['1'],
    'Diamond Max Count' : ['25'],
    'Ring Gross Weight in Grams' : ['6.60'],
    'Overall Condition' : ['Very Good'],}

dataframe = pd.DataFrame.from_dict(Diamond_Rings_data)


In [53]:
dataframe.head()

,Ring Metal,Diamonds shape in each Ring,Gems shape if any,Measurments in mm,Diamond Weights in Carats,Gem Weight in Carat,Diamond Avg Color,Diamond Avg Clarity,Gem Type if any,Gem Average Clarity,Gem Min Count,Gem Max Count,Diamond Min Count,Diamond Max Count,Ring Gross Weight in Grams,Overall Condition
0,Platinum,"emerald-cut diamond, triangular-shaped diamonds",None,"11.48 , 7.29 , 4.08","3.09 , 1.50",None,G-H-I,SI,None,None,None,None,1,25,6.60,Very Good


In [62]:
Diamond_Rings_data1 = {'Ring Metal' : ['Platinum, Gold Ring'],
    'Diamonds shape in each Ring' : ['full-cut diamonds, baguette-cut diamonds'],
    'Gems shape if any' :['cushion-shaped ruby'],
    'Measurments in mm' : ['10.10 , 9.60 , 4.00'],
    'Diamond Weights in Carats' : ['1.55, 1.20'],
    'Gem Weight in Carat' : ['3.60'],
    'Diamond Avg Color' : [' G-H-I'],
    'Diamond Avg Clarity' : ['VS'],
    'Gem Type if any' : ['Ruby'],
    'Gem Average Clarity' : ['Good'],
    'Gem Min Count': ['1'],
    'Gem Max Count': ['25'],
    'Diamond Min Count' : ['26'],
    'Diamond Max Count' : ['50'],
    'Ring Gross Weight in Grams' : ['13.75'],
    'Overall Condition' : ['Very Good'],}

dataframe1 = pd.DataFrame.from_dict(Diamond_Rings_data1)

In [63]:
dataframe.append(dataframe1)

,Ring Metal,Diamonds shape in each Ring,Gems shape if any,Measurments in mm,Diamond Weights in Carats,Gem Weight in Carat,Diamond Avg Color,Diamond Avg Clarity,Gem Type if any,Gem Average Clarity,Gem Min Count,Gem Max Count,Diamond Min Count,Diamond Max Count,Ring Gross Weight in Grams,Overall Condition
0,Platinum,"emerald-cut diamond, triangular-shaped diamonds",None,"11.48 , 7.29 , 4.08","3.09 , 1.50",None,G-H-I,SI,None,None,None,None,1,25,6.60,Very Good
0,"Platinum, Gold Ring","full-cut diamonds, baguette-cut diamonds",cushion-shaped ruby,"10.10 , 9.60 , 4.00","1.55, 1.20",3.60,G-H-I,VS,Ruby,Good,1,25,26,50,13.75,Very Good


In [67]:
m = pf['Condition Report']

In [69]:
m = m.str.split('\n')

In [70]:
m[1110]

['Condition Report*:',
 'Diamond Average Color: G-H-I',
 'Diamond Average Clarity: VS',
 'Diamond Shapes: Full-cut, Tapered Baguette-cut',
 'Diamond Count: 26 - 50',
 'Gem Type: Ruby',
 'Gem Average Clarity: Good',
 'Gem Shape: Cushion-shaped',
 'Gem Count: 1 - 25',
 'Overall Condition: Very Good',
 'Notes: Wonderful contrast between the ruby and diamonds.']

In [259]:
def get_carats(x):
    for item in x:
        if 'measuring' in item and 'carats' in item:
            mask = item.split('measuring')[1]
            mask = mask.split('carats')[0]
            if 'approximately' in mask:
                mask = mask.split('approximately')[1]
                return mask
            elif 'weighing' in mask:
                mask = mask.split('weighing')[1]
                return mask
            elif 'weighs' in mask:
                mask = mask.split('weighs')[1]
                return mask
            else:
                pass
            return mask
        elif 'measuring' in item and 'carat' in item:
            mask = item.split('measuring')[1]
            mask = mask.split('carat')[0]
            if 'approximately' in mask:
                mask = mask.split('approximately')[1]
                return mask
            # else:
            #     # pass
            elif 'weighing' in mask:
                mask = mask.split('weighing')[1]
                return mask
            else:
                pass
            return mask
        else:
            pass

In [302]:
diamond_carats = m3.apply(get_carats)

In [303]:
diamond_carats =pd.to_numeric(carats, errors='coerce')
diamond_carats.head()

0    3.45
1    5.01
2    6.40
3    7.15
4    5.04
Name: Description, dtype: float64

In [320]:
processed_diamond_ring_data = pd.DataFrame(columns=['Ring Price', 'Ring Gross Weight', 'Ring Size', 'Diamond Weight/Carats', 'Diamond Count' ])


In [321]:
processed_diamond_ring_data ['Ring Price']= diamond_ring_price
processed_diamond_ring_data ['Ring Gross Weight']= ring_gross_weight
processed_diamond_ring_data ['Ring Size'] = ring_size
processed_diamond_ring_data ['Diamond Weight/Carats'] = diamond_carats
processed_diamond_ring_data ['Diamond Count'] = diamond_count
processed_diamond_ring_data.head()


,Ring Price,Ring Gross Weight,Ring Size,Diamond Weight/Carats,Diamond Count
0,4000000.0,5.10,6.0,3.45,25.0
1,4000000.0,6.90,6.0,5.01,50.0
2,4000000.0,4.47,4.0,6.40,25.0
3,4000000.0,5.51,6.0,7.15,25.0
4,4000000.0,5.90,6.0,5.04,25.0


In [322]:
# processed_diamond_ring_data.to_csv('processed_diamond_ring_data.csv')